In [1]:
from prisma import Prisma
import pandas as pd
from datetime import datetime
import json

db = Prisma()
await db.connect()

In [2]:
from pymongo import MongoClient

# Provide the mongodb atlas url to connect python to mongodb using pymongo
CONNECTION_STRING = "mongodb://localhost:27017"

# Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
client = MongoClient(CONNECTION_STRING)

# Create the database for our example (we will use the same database throughout the tutorial
mongo_db = client["eezeeDb"]

product_collection = mongo_db["products"]

category_colletion = mongo_db["categories"]

In [3]:
stats = {}

In [4]:
all_items_count = await db.item.count(
    where={
        "deleted": False,
        "baseCurrency": "SGD",
        "isHiddenFromSearchEngine": False
    }
)
stats["all_items_count"] = all_items_count
stats

{'all_items_count': 354945}

In [5]:
cte_query = """
WITH spec AS (
    select 
        itemspec.*,
        specheader.id as "itemCategorySpecHeaderId",
        specheader.*
    from "ItemSpecification" itemspec
    join "ItemCategorySpecHeader" specheader on itemspec."itemCategorySpecHeaderId" = specheader.id
)
"""

In [6]:
items_w_specs_query = """
    select 
        COUNT(DISTINCT item.id) AS "itemWithSpecsCount"
    from "Item" item
    join spec on spec."itemId" = item.id and item."subCat" = spec."itemCategoryTitleUrl"
    where 1=1
    and item.deleted = false
    and item."baseCurrency" = 'SGD'
    and item."isHiddenFromSearchEngine" = false
    """
items_with_specs_count = await db.query_raw(query=cte_query+items_w_specs_query)

stats["product_specification"] = {"products_w_specification_count": items_with_specs_count[0]["itemWithSpecsCount"]}
stats["product_specification"]["specs_coverage"] = (items_with_specs_count[0]["itemWithSpecsCount"] / all_items_count) * 100

distinct_specs = """
    select 
        COUNT(DISTINCT spec."nameKebabCase") AS "distinctSpecs"
    from "Item" item
    join spec on spec."itemId" = item.id and item."subCat" = spec."itemCategoryTitleUrl"
    where 1=1
    and item.deleted = false
    and item."baseCurrency" = 'SGD'
    and item."isHiddenFromSearchEngine" = false
"""
response = await db.query_raw(query=cte_query+distinct_specs)
stats["product_specification"]["distinct_specifications"] = response[0]["distinctSpecs"]
stats

{'all_items_count': 354945,
 'product_specification': {'products_w_specification_count': 158385,
  'specs_coverage': 44.62240628829819,
  'distinct_specifications': 3776}}

In [7]:
items_w_brand = await db.item.count(
    where={
        "deleted": False,
        "baseCurrency": "SGD",
        "isHiddenFromSearchEngine": False,
        "brand": {
            "not": "",
        }
    }
)

stats["brand"] = {"products_w_brand_count": items_w_brand}

distinct_brands = await db.query_raw(query="""
SELECT COUNT(DISTINCT item.brand) AS "unique_brand_count"
from "Item" item
where 1=1
    and item.deleted = false
    and item."baseCurrency" = 'SGD'
    and item."isHiddenFromSearchEngine" = false
""")
stats["brand"]["distinct_brands"] = distinct_brands[0]["unique_brand_count"]
stats["brand"]["brand_coverage"] = int((items_w_brand/all_items_count) * 100)

stats

{'all_items_count': 354945,
 'product_specification': {'products_w_specification_count': 158385,
  'specs_coverage': 44.62240628829819,
  'distinct_specifications': 3776},
 'brand': {'products_w_brand_count': 354944,
  'distinct_brands': 3105,
  'brand_coverage': 99}}

In [8]:
items_w_model_number = await db.item.count(
    where={
        "deleted": False,
        "baseCurrency": "SGD",
        "isHiddenFromSearchEngine": False,
        "modelNumber": {
            "not": ""
        }
    }
)
stats["model_number"] = {"products_w_model_number_count": items_w_model_number}

stats["model_number"]["model_number_coverage"] = int((items_w_model_number/all_items_count) * 100)

stats


{'all_items_count': 354945,
 'product_specification': {'products_w_specification_count': 158385,
  'specs_coverage': 44.62240628829819,
  'distinct_specifications': 3776},
 'brand': {'products_w_brand_count': 354944,
  'distinct_brands': 3105,
  'brand_coverage': 99},
 'model_number': {'products_w_model_number_count': 354945,
  'model_number_coverage': 100}}

In [9]:
items_w_category = await db.item.count(
    where={
        "deleted": False,
        "baseCurrency": "SGD",
        "isHiddenFromSearchEngine": False,
        "subCat": {
            "not": ""
        }
    }
)

stats["category"] = {"products_w_category_count": items_w_category}
stats["category"]["category_coverage"] = int((items_w_category/all_items_count) * 100)
distinct_categories = await db.query_raw(query="""
SELECT COUNT(DISTINCT item."subCat") AS "distinct_cat_count"
from "Item" item
where 1=1
    and item.deleted = false
    and item."baseCurrency" = 'SGD'
    and item."isHiddenFromSearchEngine" = false
""")
stats["category"]["distinct_categories"] = distinct_categories[0]["distinct_cat_count"]


stats

{'all_items_count': 354945,
 'product_specification': {'products_w_specification_count': 158385,
  'specs_coverage': 44.62240628829819,
  'distinct_specifications': 3776},
 'brand': {'products_w_brand_count': 354944,
  'distinct_brands': 3105,
  'brand_coverage': 99},
 'model_number': {'products_w_model_number_count': 354945,
  'model_number_coverage': 100},
 'category': {'products_w_category_count': 354945,
  'category_coverage': 100,
  'distinct_categories': 1465}}

In [10]:
print(json.dumps(stats, indent=2))

{
  "all_items_count": 354945,
  "product_specification": {
    "products_w_specification_count": 158385,
    "specs_coverage": 44.62240628829819,
    "distinct_specifications": 3776
  },
  "brand": {
    "products_w_brand_count": 354944,
    "distinct_brands": 3105,
    "brand_coverage": 99
  },
  "model_number": {
    "products_w_model_number_count": 354945,
    "model_number_coverage": 100
  },
  "category": {
    "products_w_category_count": 354945,
    "category_coverage": 100,
    "distinct_categories": 1465
  }
}


In [11]:
sample_query = """select 
    DISTINCT item.id
from "Item" item
join spec on spec."itemId" = item.id and item."subCat" = spec."itemCategoryTitleUrl"
where 1=1
and item.deleted = false
and item."baseCurrency" = 'SGD'
LIMIT 5
"""

response = await db.query_raw(query=cte_query+sample_query)
item_ids = [item["id"] for item in response]


In [12]:
items = await db.item.find_many(
    where={
        "id": {
            "in": item_ids
        }
    },
    include={
        "ItemCategory": True,
        "ItemSpecifications": {
            "include": {
                "ItemCategorySpecHeader": True
            }
        }
    }
)

item_json = [item.model_dump() for item in items]


In [19]:
df = pd.DataFrame(item_json)
df = df.drop(["deleted", "sellerId", "updatedAt", "supplierSku", "isHiddenFromSearchEngine", "ItemInCatalogs", "sku"], axis=1)
df.head(5)

,createdAt,id,title,brand,mainCat,baseCurrency,modelNumber,subCat,ItemSpecifications,ItemCategory
0,2022-08-28 18:27:15.466000+00:00,59f75a869518e2289998f32f,ORING AS568 AS002 ID 1.07 X CS 1.27 Viton Fluo...,oem,mechanical-parts,SGD,ORING AS568 AS002 ID 1.07 X CS 1.27 Viton Fluo...,o-ring,"[{'id': 2010631, 'createdAt': 2023-11-06 04:00...",{'createdAt': 2023-04-12 16:21:06.290000+00:00...
1,2022-08-28 18:27:15.466000+00:00,59f75af79518e2289998f342,ORING AS568 AS002 ID 1.07 X CS 1.27 Nitrile (N...,oem,mechanical-parts,SGD,ORING AS568 AS002 ID 1.07 X CS 1.27 Nitrile (N...,o-ring,"[{'id': 1993425, 'createdAt': 2023-11-06 03:59...",{'createdAt': 2023-04-12 16:21:06.290000+00:00...
2,2022-08-28 18:27:15.466000+00:00,59f75b789518e2289998f35a,ORING AS568 AS002 ID 1.07 X CS 1.27 Nitrile (N...,oem,mechanical-parts,SGD,ORING AS568 AS002 ID 1.07 X CS 1.27 Nitrile (N...,o-ring,"[{'id': 2002972, 'createdAt': 2023-11-06 03:59...",{'createdAt': 2023-04-12 16:21:06.290000+00:00...
3,2022-08-28 18:27:15.466000+00:00,59f75d2d9518e2289998f3ce,ORING AS568 AS002 ID 1.07 X CS 1.27 Ethylene P...,oem,mechanical-parts,SGD,ORING AS568 AS002 ID 1.07 X CS 1.27 Ethylene P...,o-ring,"[{'id': 2022520, 'createdAt': 2023-11-06 04:00...",{'createdAt': 2023-04-12 16:21:06.290000+00:00...
4,2022-08-28 18:27:15.466000+00:00,59f75da29518e2289998f407,ORING AS568 AS002 ID 1.07 X CS 1.27 Silicone 7...,oem,mechanical-parts,SGD,ORING AS568 AS002 ID 1.07 X CS 1.27 Silicone 7...,o-ring,"[{'id': 2010406, 'createdAt': 2023-11-06 04:00...",{'createdAt': 2023-04-12 16:21:06.290000+00:00...
